#### Simple Chatbot for GPT-4

Load the API key and relevant Python libraries

In [ ]:
import os
import openai

In [ ]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ.get("OPENAI_API_KEY")

Helper function

In [ ]:
def get_completion_and_tokens(
    messages,
    model="gpt-4",
    temperature=0,
    max_tokens=500,
):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )

    content = response.choices[0].message["content"]

    token_dict = {
        "prompt_tokens": response["usage"]["prompt_tokens"],
        "completion_tokens": response["usage"]["completion_tokens"],
        "total_tokens": response["usage"]["total_tokens"],
    }

    return content, token_dict

Messages to GPT

In [ ]:
messages = [
    {
        "role": "system",
        "content": """You are a large language model that has been trained as an expert software engineer Answer all questions as best as you can.""",
    },
    {
        "role": "user",
        "content": """Write 5 python print methods that return 10 sentences ending with the word 'apple'.""",
    },
]

Get response from GPT, and print and save

In [ ]:
response, token_dict = get_completion_and_tokens(messages)

print(response)
print(token_dict)

def save_to_file(messages, response, token_dict, filename='chats.txt'):
    with open(filename, 'a') as f:
        for message in messages:
            if message['role'] == 'user':
                f.write('User: ' + message['content'] + '\n')
            elif message['role'] == 'system':
                f.write('System: ' + message['content'] + '\n')

        f.write('\n')
        f.write('Response: ' '\n' + str(response) + '\n')
        f.write('\n')
        f.write('Token Dict: ' + str(token_dict) + '\n')
        f.write('\n' + '--------------------------------------------' + '\n')
        f.write('\n')

save_to_file(messages, response, token_dict)